## Install and Import Libraries

In [ ]:
!pip install transformers pyvis

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 756.0/756.0 kB 3.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 21.7 MB/s eta 0:00:00
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/pip/_vendor/pkg_resources/__init__.py", line 3108, in _dep_map
    return self.__dep_map
  File "/usr/local/lib/python3.10/dist-packages/pip/_vendor/pkg_resources/__init__.py", line 2901, in __getattr__
    raise AttributeError(attr)
AttributeError: _DistInfoDistribution__dep_map

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/pip/_internal/cli/base_command.py", line 169, in exc_logging_wrapper
    status = run_func(*args)
  File "/usr/local/lib/python3.10/dist-packages/pip/_internal/cli/req_command.py", line 242, in wrapper
    return func(self, options, args)
  File "/usr/local/lib/python3.10/dist-packages/pip/_internal/commands/install.py",

In [ ]:
#from transformers import AutoModelForSeq2SeqLM, AutoTokenizer
import math
import torch
import pandas as pd
import json
from typing_extensions import DefaultDict
#from pyvis.network import Network
import IPython

##Read From CSV



In [ ]:
df = pd.read_csv("harry_potter_books.csv")

In [ ]:
books = set(df['book'])

def extract_text(title):
  contents = ''
  temp_book = df[df['book'] == title]['text']
  for i in temp_book:
    contents += i
  return contents


## Load the Relation Extraction Model

In [ ]:
# Load model and tokenizer
tokenizer = AutoTokenizer.from_pretrained("Babelscape/rebel-large")
model = AutoModelForSeq2SeqLM.from_pretrained("Babelscape/rebel-large")

## From Long Text to KB

In [ ]:
class KB():
    def __init__(self):
        self.relations = []

    def text_normalize(self,text):
        text = text.lower()
        text = text.replace("'s","")
        return text

    def are_relations_equal(self, r1, r2):
        return all(r1[attr] == r2[attr] for attr in ["head", "type", "tail"])

    def exists_relation(self, r1):
        return any(self.are_relations_equal(r1, r2) for r2 in self.relations)

    def merge_relations(self, r1):
        for r2 in self.relations:
            if self.are_relations_equal(r1, r2):
                # Merge spans
                spans_to_add = [span for span in r1["meta"]["spans"] if span not in r2["meta"]["spans"]]
                r2["meta"]["spans"] += spans_to_add
                # Increment the count of occurrences
                r2["count"] += 1
                break

    def add_relation(self, r):
        r['head'] = self.text_normalize(r['head'])
        r['tail'] = self.text_normalize(r['tail'])
        r['count'] = 1

        if not self.exists_relation(r):
            self.relations.append(r)
        else:
            self.merge_relations(r)

    def merge_kb(self, other_kb):
        for relation in other_kb.relations:
            self.add_relation(relation)

    def print(self):
        print("Relations:")
        for r in self.relations:
            print(f"  {r}")


In [ ]:
def extract_relations_from_model_output(text):
    relations = []
    relation, subject, relation, object_ = '', '', '', ''
    text = text.strip()
    current = 'x'
    text_replaced = text.replace("<s>", "").replace("<pad>", "").replace("</s>", "")
    for token in text_replaced.split():
        if token == "<triplet>":
            current = 't'
            if relation != '':
                relations.append({
                    'head': subject.strip(),
                    'type': relation.strip(),
                    'tail': object_.strip()
                })
                relation = ''
            subject = ''
        elif token == "<subj>":
            current = 's'
            if relation != '':
                relations.append({
                    'head': subject.strip(),
                    'type': relation.strip(),
                    'tail': object_.strip()
                })
            object_ = ''
        elif token == "<obj>":
            current = 'o'
            relation = ''
        else:
            if current == 't':
                subject += ' ' + token
            elif current == 's':
                object_ += ' ' + token
            elif current == 'o':
                relation += ' ' + token
    if subject != '' and relation != '' and object_ != '':
        relations.append({
            'head': subject.strip(),
            'type': relation.strip(),
            'tail': object_.strip()
        })
    return relations

In [ ]:
# extract relations for each span and put them together in a knowledge base
def from_text_to_kb(text, span_length=128, verbose=False):
    # tokenize whole text
    inputs = tokenizer([text], return_tensors="pt")

    # compute span boundaries
    num_tokens = len(inputs["input_ids"][0])
    if verbose:
        print(f"Input has {num_tokens} tokens")
    num_spans = math.ceil(num_tokens / span_length)
    if verbose:
        print(f"Input has {num_spans} spans")
    overlap = math.ceil((num_spans * span_length - num_tokens) /
                        max(num_spans - 1, 1))
    spans_boundaries = []
    start = 0
    for i in range(num_spans):
        spans_boundaries.append([start + span_length * i,
                                 start + span_length * (i + 1)])
        start -= overlap
    if verbose:
        print(f"Span boundaries are {spans_boundaries}")

    # transform input with spans
    tensor_ids = [inputs["input_ids"][0][boundary[0]:boundary[1]]
                  for boundary in spans_boundaries]
    tensor_masks = [inputs["attention_mask"][0][boundary[0]:boundary[1]]
                    for boundary in spans_boundaries]
    inputs = {
        "input_ids": torch.stack(tensor_ids),
        "attention_mask": torch.stack(tensor_masks)
    }

    # generate relations
    num_return_sequences = 3
    gen_kwargs = {
        "max_length": 256,
        "length_penalty": 0,
        "num_beams": 3,
        "num_return_sequences": num_return_sequences
    }
    generated_tokens = model.generate(
        **inputs,
        **gen_kwargs,
    )

    # decode relations
    decoded_preds = tokenizer.batch_decode(generated_tokens,
                                           skip_special_tokens=False)

    # create kb
    kb = KB()
    i = 0
    for sentence_pred in decoded_preds:
        current_span_index = i // num_return_sequences
        relations = extract_relations_from_model_output(sentence_pred)
        for relation in relations:
            relation["meta"] = {
                "spans": [spans_boundaries[current_span_index]]
            }
            kb.add_relation(relation)
        i += 1

    return kb

In [ ]:

for ch in books:
  text = extract_text(ch)
  kb = None
  for i in range(0,len(text)-1000,1000):
    if not kb:
      kb = from_text_to_kb(text[i:i+1000])
      kb.print()
    else:
      new = from_text_to_kb(text[i:i+1000])
      kb.merge_kb(new)
      kb.print()

  json_data = json.dumps(kb.relations, indent=4)



Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Relations:
  {'head': 'riddle house', 'type': 'located in the administrative territorial entity', 'tail': 'little hangleron', 'meta': {'spans': [[0, 128]]}, 'count': 1}
  {'head': 'little hangleron', 'type': 'instance of', 'tail': 'village', 'meta': {'spans': [[0, 128]]}, 'count': 1}
  {'head': 'the riddle house', 'type': 'located in the administrative territorial entity', 'tail': 'little hangleron', 'meta': {'spans': [[0, 128]]}, 'count': 1}
  {'head': 'riddlehouse', 'type': 'present in work', 'tail': 'the little hagletons', 'meta': {'spans': [[99, 227]]}, 'count': 1}
  {'head': 'riddlehouse', 'type': 'occupant', 'tail': 'little hagletons', 'meta': {'spans': [[99, 227]]}, 'count': 1}
  {'head': 'riddlehouse', 'type': 'located in the administrative territorial entity', 'tail': 'little haglet', 'meta': {'spans': [[99, 227]]}, 'count': 1}
Relations:
  {'head': 

## Entity Normalization and Remove Redundant Relationships

In [ ]:
def read_relations(file_path):
  with open(file_path, 'r') as file:
    relations = json.load(file)
  return relations

In [ ]:
def entity_normalization(relations):
  def extract_synonyms(relations,synonyms):
      if relations['type'] in ['subclass of','instance of']:
          synonyms[relations['tail']].append(relations['head'])

  def build_entity_syn(relations,syn):
    synonyms = DefaultDict(list)
    for row in relations:
      extract_synonyms(row,synonyms)
    reverse_synonyms = {}
    for key, values in synonyms.items():
        for value in values:
            reverse_synonyms[value] = key
    return reverse_synonyms

  def replace_entity_and_build_syn_connection(relations,syn):
    for rel in relations:
      head = rel['head']
      if head in syn:
        rel['head'] = syn[head]

    for key,value in syn.items():
      relations.append({
          'head': key,
          'type': 'subclass of',
          'tail': value,
          'meta':{
              'type':'synonyms'
          },
          'count': 1
      })
    return relations
  syn = build_entity_syn(relations)
  file = replace_entity_and_build_syn_connection(relations,syn)
  return file

In [ ]:
def preprocess_json(file_name):
  file = read_relations(file_name)
  file = entity_normalization(file)
  return file


## hand craft to normalize

In [ ]:
file = read_relations("ch1-ch6 merged.json")

In [ ]:
entity_synonyms = {
    "harry potter": ["the boy who lived", "the chosen one", "'arry potter", "harry", "potter","'arry potter"],
    "hermione granger": ["hermione", "miss granger", "granger"],
    "ron weasley": ["ron", "ronald weasley", "weasley"],
    "albus dumbledore": ["dumbledore", "professor dumbledore", "albus"],
    "severus snape": ["snape", "professor snape", "severus"],
    "voldemort": ["he-who-must-not-be-named", "you-know-who", "the dark lord", "tom riddle", "lord voldemort"],
    "hagrid": ["rubeus hagrid", "'agrid", "gamekeeper"],
    "draco malfoy": ["draco", "malfoy"],
    "sirius black": ["sirius", "padfoot", "black"],
    "remus lupin": ["lupin", "moony", "professor lupin"],
    "peter pettigrew": ["wormtail", "pettigrew"],
    "ginny weasley": ["ginny", "ginevra weasley"],
    "neville longbottom": ["neville", "longbottom"],
    "luna lovegood": ["luna", "'loony' lovegood"],
    "dobby": ["the house elf", "dobby", "free elf"],
    "hogwarts": ["hogwarts school of witchcraft and wizardry", "school of witchcraft and wizardry"],
    "quidditch": ["the sport of wizards", "wizarding sport"],
    "azkaban": ["the wizard prison", "wizard prison"],
    "minerva mcgonagall": ["mcgonagall", "professor mcgonagall"],
    "mad-eye moody": ["mad-eye", "alastor moody", "moody"],
    "bellatrix lestrange": ["bellatrix", "lestrange"],
    "dolores umbridge": ["umbridge", "professor umbridge"],
    "lucius malfoy": ["lucius", "mr. malfoy"],
    "molly weasley": ["molly", "mrs. weasley"],
    "arthur weasley": ["arthur", "mr. weasley"],
    "fred and george weasley": ["fred weasley", "george weasley", "the twins", "weasley twins"],
    "cedric diggory": ["cedric", "diggory"],
    "fleur delacour": ["fleur", "madame maxime"],
    "dumbledore's army": ["da", "the army"],
    "order of the phoenix": ["the order"],
    "the death eaters": ["death eaters"],
    "the triwizard tournament": ["triwizard tournament"],
    "the prophecy": ["prophecy"],
    "the elder wand": ["elder wand"],
    "the invisibility cloak": ["invisibility cloak"],
    "the resurrection stone": ["resurrection stone"],
    "the marauder's map": ["marauder's map", "map"]
}
reverse_synonyms = {}
for key, values in entity_synonyms.items():
    for value in values:
        reverse_synonyms[value] = key

In [ ]:
def process_and_aggregate(file, reverse_synonyms):
    processed = {}
    for row in file:
        # Normalize head and tail by stripping spaces and converting to lower case
        head = reverse_synonyms.get(row['head'], row['head'])
        tail = reverse_synonyms.get(row['tail'], row['tail'])
        if len(head) > 20 or len(tail) > 20:
            continue
        if '*' in head or '-' in head or '*' in tail or '=' in tail or ',' in head or ',' in tail:
            continue
        # Continue if head and tail are the same
        if head == tail or head.strip() == tail.strip():
            continue

        rel = row['type']

        key = (head, rel, tail)

        if key in processed:
            processed[key]['count'] += row['count']
        else:
            processed[key] = {
                'head': head,
                'type': rel,
                'tail': tail,
                'count': row['count']
            }


    return list(processed.values())

file = process_and_aggregate(file, reverse_synonyms)


In [ ]:
with open('proce.json', 'w') as f:
    json.dump(file, f)

## Visualize KB

In [ ]:
def create_and_show_network(relations):
    # Initialize a network graph
    net = Network(directed=True, height="100vh", width="100%", bgcolor="#222222", font_color="white")

    # Add nodes and edges based on the relations data
    for relation in relations:
        net.add_node(relation['head'], label=relation['head'], title=relation['head'])
        net.add_node(relation['tail'], label=relation['tail'], title=relation['tail'])
        net.add_edge(relation['head'], relation['tail'], title=relation['type'])

    # Define graph aesthetics and physics for better visualization
    net.set_options("""
    {
      "nodes": {
        "font": {
          "size": 12,
          "color": "#ffffff"
        }
      },
      "edges": {
        "arrows": {
          "to": {
            "enabled": true,
            "scaleFactor": 0.5
          }
        },
        "color": {
          "inherit": true
        },
        "smooth": {
          "type": "dynamic"
        }
      },
      "physics": {
        "forceAtlas2Based": {
          "gravitationalConstant": -26,
          "centralGravity": 0.005,
          "springLength": 230,
          "springConstant": 0.18
        },
        "maxVelocity": 146,
        "minVelocity": 0.1,
        "solver": "forceAtlas2Based",
        "timestep": 0.35,
        "stabilization": { "iterations": 150 }
      }
    }
    """)

    # Save and display the network
    net.show("relations_network.html", notebook = False)
    files.download('relations_network.html')





In [ ]:
# Call the function to create and show the network
create_and_show_network(kb.relations)